# Create My Own Sound Collection

In this notebook, we will build our own collection of sound fonts by querying the Freesound API. We will use modular code stored in the **src** folder to:
- Query Freesound for sounds matching our desired keywords.
- Download high-quality previews.
- Store relevant metadata in a CSV file.

This approach follows the structure of the original Notebook 1 from the AMPLAB module, our implementation is modular.



## Environment Setup and Imports

This cell confirms that our configuration (API key, file paths, etc.) is loaded correctly from src/config.py and that our functions for querying Freesound are accessible.

In [9]:
# Ensure that your virtual environment is activated and dependencies are installed (see requirements.txt).

# Import configuration settings and utility modules from src
import os
import pandas as pd
import sys

# Añadir la ruta absoluta del directorio raíz del proyecto
sys.path.append(os.path.abspath(".."))

# Our configuration file contains API keys, file paths, etc.
from src import config  
# Our sound_collection module includes functions to query Freesound and process sound records.
from src.sound_collection import query_freesound, download_sound_preview, make_metadata_record

# Display configuration to confirm settings
print("Freesound API Key:", config.FREESOUND_API_KEY)
print("Files Directory:", config.RAW_DIR)
print("Metadata CSV File:", config.DATAFRAME_FILENAME)


Freesound API Key: zr5kUObkkKtoSIiWGRPG6DPNUMOxdU1ercdOGcaJ
Files Directory: ../data/raw
Metadata CSV File: ../data/metadata/fonts_collection.csv


## Prepare the Data Directory (Code)

In [ ]:
# Check if the directory to store downloaded sound files exists; if not, create it.
for folder in [config.RAW_DIR, config.PROCESSED_DIR, config.METADATA_DIR]:
    if not os.path.exists(folder):
        os.makedirs(folder)
        print(f"Created directory: {folder}")
    else:
        print(f"Directory {folder} already exists.")


Directory ../data/raw already exists.
Directory ../data/processed already exists.
Directory ../data/metadata already exists.


## Define Freesound Query Parameters

Adjust the queries to target the type of sound “fonts” you want. For example, “dog bark” might be used as a percussive element, while “vowels” can add a human quality. The filters ensure that only sounds with appropriate durations are retrieved.

In [11]:
# Define a list of queries to build our sound collection.
# Here we customize our search terms and filters based on our creative vision.
freesound_queries = [
    {
        'query': 'dog bark',
        'filter': None,  # Using default filter (duration: 0-30 sec)
        'num_results': 20,
    },
    {
        'query': 'percussion sounds',
        'filter': 'duration:[0 TO 1]',  # Very short percussive sounds
        'num_results': 20,
    },
    {
        'query': 'vowels',
        'filter': 'duration:[1 TO 2]',  # Vocal sounds with a bit longer duration
        'num_results': 20,
    },
]

# Display the queries for confirmation.
print("Freesound Queries:")
for q in freesound_queries:
    print(q)


Freesound Queries:
{'query': 'dog bark', 'filter': None, 'num_results': 20}
{'query': 'percussion sounds', 'filter': 'duration:[0 TO 1]', 'num_results': 20}
{'query': 'vowels', 'filter': 'duration:[1 TO 2]', 'num_results': 20}


## Query Freesound and Collect Sound Objects 

This cell uses our query_freesound function to perform the API searches. The results from each query are concatenated into one list representing our overall collection.

In [12]:
# Initialize an empty list to store all retrieved sound objects.
all_sounds = []

# Loop through each query configuration and perform the Freesound search.
for query_info in freesound_queries:
    results = query_freesound(query_info['query'], query_info['filter'], query_info['num_results'])
    print(f"Retrieved {len(results)} sounds for query '{query_info['query']}'.")
    all_sounds.extend(results)

print(f"Total sounds retrieved: {len(all_sounds)}")


Retrieved 20 sounds for query 'dog bark'.
Retrieved 20 sounds for query 'percussion sounds'.
Retrieved 17 sounds for query 'vowels'.
Total sounds retrieved: 57


## Download Sound Previews

In [13]:
# Loop through each sound and download its preview to our designated directory.
for idx, sound in enumerate(all_sounds):
    print(f"Downloading sound {idx+1}/{len(all_sounds)}: id {sound.id}")
    download_sound_preview(sound, config.RAW_DIR)


## Create Metadata Records and Save DataFrame

The make_metadata_record function extracts key details from each sound, and we then store the entire collection in a CSV file. This DataFrame will serve as the basis for further analysis and later stages of the project.

In [14]:
# Build a list of metadata records for each sound using our utility function.
metadata_records = [make_metadata_record(sound, config.RAW_DIR) for sound in all_sounds]

# Create a Pandas DataFrame from the metadata records.
df_metadata = pd.DataFrame(metadata_records)

# Save the DataFrame to CSV for later use.
df_metadata.to_csv(config.DATAFRAME_FILENAME, index=False)
print(f"Saved metadata DataFrame with {len(df_metadata)} entries to {config.DATAFRAME_FILENAME}.")


Saved metadata DataFrame with 57 entries to ../data/metadata/fonts_collection.csv.


## Display the Metadata DataFrame

Displaying the DataFrame allows you to visually inspect the metadata and confirm that all information (e.g., sound names, tags, file paths) has been captured correctly.

In [15]:
# Load the metadata DataFrame to verify its contents.
df_loaded = pd.read_csv(config.DATAFRAME_FILENAME)
display(df_loaded)


,name,username,license,tags,freesound_id,path
0,Jazz the Dog Howl & Bark (019).wav,delphidebrain,https://creativecommons.org/licenses/by-nc/4.0/,"['Barking', 'Jazz', 'Barks', 'Dog', 'Howling',...",236125,../data/raw/236125_180659-hq.ogg
1,Animal Dog Bark 08.wav,abhisheky948,http://creativecommons.org/publicdomain/zero/1.0/,"['Bark', 'Dog', 'Animal']",625497,../data/raw/625497_13366994-hq.ogg
2,single dog bark 2,crazymonke9,https://creativecommons.org/licenses/by/4.0/,"['growling', 'growl', 'barking', 'pet', 'dog',...",418106,../data/raw/418106_6078577-hq.ogg
3,Dogs bark and howl,Weak_Hero,http://creativecommons.org/publicdomain/zero/1.0/,"['barking', 'whine', 'dog', 'bark', 'puppy', '...",612856,../data/raw/612856_13194336-hq.ogg
4,Barking Inez 2,wateenfotos,http://creativecommons.org/publicdomain/zero/1.0/,"['dogs', 'dog', 'barking', 'woof', 'bark', 'in...",749351,../data/raw/749351_16213215-hq.ogg
5,Small Dog Bark 02,noctaro,https://creativecommons.org/licenses/by/4.0/,"['pet', 'bark', 'barking', 'animal', 'dog', 'p...",242413,../data/raw/242413_4165591-hq.ogg
6,Some dogs barking close to a road in Anatolia ...,felix.blume,http://creativecommons.org/publicdomain/zero/1.0/,"['a', 'Anatolia', 'Blume', 'barking', 'Turkey'...",166489,../data/raw/166489_1661766-hq.ogg
7,Barking Dog,SuperStudioBR,http://creativecommons.org/publicdomain/zero/1.0/,"['bark', 'happy', 'loud', 'dog', 'barking']",180977,../data/raw/180977_3370987-hq.ogg
8,Dog bark 1,jorickhoofd,https://creativecommons.org/licenses/by/4.0/,"['average-size', 'bark', 'Dog']",160092,../data/raw/160092_2888453-hq.ogg
9,Dog Barking Distance - 1.wav,SpaceJoe,http://creativecommons.org/publicdomain/zero/1.0/,"['distance', 'barking', 'dog', 'woof', 'attack...",485962,../data/raw/485962_6150892-hq.ogg
